In [2]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import functools

data_path = 'Data/'

### Loading in test data
load data
organise info
clean up columns

In [3]:
choices = pd.read_csv(data_path + 'responses.csv')

In [4]:
agenda = [re.findall(r'\[([^\]]+)', col) for col in choices.columns[2:]]

In [5]:
new_cols = ['timestamp','name'] + [re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_') for co in agenda]

In [6]:
choices.columns = new_cols

In [7]:
schedule = [(co[0].replace(' ', '_'), re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_')) for co in agenda]

In [8]:
schedule_dict = {}
for pair in schedule:
    if pair[0] not in schedule_dict.keys():
        schedule_dict[pair[0]] = []
    schedule_dict[pair[0]].append(pair[1])

In [9]:
company_avai_dict_og = {re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_'):int(co[1].split('[')[1]) for co in agenda}
company_avai_dict = company_avai_dict_og.copy()

### Random allocation

Create dictionary that matches time slots to choices

In [10]:
choices = choices.drop(['timestamp'], axis = 1)

Trying out allocation

In [11]:
# dropping NA
choices = choices[~choices.name.isnull()].reset_index(drop = True)

In [12]:
def prize_draw(co, level, pool):
    limit = company_avai_dict[co]
    new_pool = list(set(choices.name[choices[co] == level])&set(pool))
    if len(new_pool) > limit:
        return(list(np.random.choice(new_pool, limit, replace=False)))
    else:
        return(new_pool)

In [13]:
company_avai_dict = company_avai_dict_og
prize_draw_dict = {}
for slot, companies in schedule_dict.items():
    print('------------new time slot-------------')
    print(companies)
    pool = list(choices.name)
    all_levels = range(1,len(companies)+1)
    print(all_levels)
    for level in all_levels:
        for co in companies:
            print(co, level)
            if co not in prize_draw_dict.keys():
                prize_draw_dict[co] = []
            winners = prize_draw(co, level, pool)
            company_avai_dict[co] = company_avai_dict[co] - len(winners)
            prize_draw_dict[co].extend(winners)
            pool = [person for person in pool if person not in winners]
            print('there remains', len(pool), ' to be distributed')
            if company_avai_dict[co] == 0:
                print('company exhausted')
            if len(pool) == 0:
                print('pool exhausted')

------------new time slot-------------
['Neuroprofiler', 'Regnosys', 'AQR']
range(1, 4)
Neuroprofiler 1
there remains 30  to be distributed
company exhausted
Regnosys 1
there remains 23  to be distributed
AQR 1
there remains 8  to be distributed
company exhausted
Neuroprofiler 2
there remains 8  to be distributed
company exhausted
Regnosys 2
there remains 8  to be distributed
AQR 2
there remains 8  to be distributed
company exhausted
Neuroprofiler 3
there remains 8  to be distributed
company exhausted
Regnosys 3
there remains 0  to be distributed
company exhausted
pool exhausted
AQR 3
there remains 0  to be distributed
company exhausted
pool exhausted
------------new time slot-------------
['Google_Deepmind', 'Blockchain', 'Pierre_Richemond', 'Stéphane_Leroy', 'King.com', 'BNP_Paribas_Data_Analytics', 'Circle']
range(1, 8)
Google_Deepmind 1
there remains 37  to be distributed
company exhausted
Blockchain 1
there remains 37  to be distributed
Pierre_Richemond 1
there remains 35  to be d

In [14]:
for slot, companies in schedule_dict.items():
    total = sum([len(prize_draw_dict[company]) for company in companies])
    print('for', slot, total, ' have been allocated')

for TUESDAY_AFTERNOON 45  have been allocated
for WEDNESDAY_LUNCH 45  have been allocated
for WEDNESDAY_AFTERNOON 45  have been allocated
for THURSDAY_MORNING 45  have been allocated
for THURSDAY_AFTERNOON 45  have been allocated


In [15]:
len(choices)

45

it seems as though we're good

### Save as xlsx

In [16]:
pd.DataFrame.from_dict(prize_draw_dict, orient='index').transpose().to_excel(data_path+'prize_draw_res.xlsx')